# Use Case: ZAF043 Crowd Anomaly Detection

## 1 - Load the data

In [1]:
import os
import cv2
import numpy as np

def load_training_data(directory):
    # Initialize lists for frames and labels
    frames = []
    labels = []

    # Iterate over the files in the directory
    for filename in os.listdir(directory):
        # Get the full file path
        file_path = os.path.join(directory, filename)

        # Load the frame using OpenCV
        frame = cv2.imread(file_path)

        # Preprocess the frame if needed

        # Append the frame to the frames list
        frames.append(frame)

        # Extract the label from the filename (assuming the filename contains the label information)
        label = filename.split("_")[0]  # Extract the label from the filename based on the naming convention
        labels.append(label)

    # Convert frames and labels to numpy arrays
    frames = np.array(frames)
    labels = np.array(labels)

    return frames, labels

# Directory path containing the training data
training_data_directory = "D:\\Data Set\\Crowd Anomaly Detection\\UCSD_Anomaly_Dataset"

# Load training data
training_frames, training_labels = load_training_data(training_data_directory)

# Print the shape of the training frames and labels
print("Training Frames Shape:", training_frames.shape)
print("Training Labels Shape:", training_labels.shape)

Training Frames Shape: (6,)
Training Labels Shape: (6,)


## 2 - Data Preprocessing, Exploratory Data Analysis

In [2]:
def extract_features(frames):
    features = []
    
    # Convert frames to grayscale
    gray_frames = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in frames]
    
    # Calculate optical flow for consecutive frames
    for i in range(len(frames) - 1):
        prev_frame = gray_frames[i]
        next_frame = gray_frames[i + 1]
        
        # Compute optical flow using Lucas-Kanade method
        flow = cv2.calcOpticalFlowFarneback(prev_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        # Extract relevant features from the optical flow
        # Example: Compute the mean and standard deviation of the flow vectors
        mean_flow = np.mean(flow)
        std_flow = np.std(flow)
        
        # Append the extracted features to the list
        features.append([mean_flow, std_flow])
    
    return np.array(features)

## 3 - Models
In this code, the train_mdt_model() function takes features (the extracted features) and num_components (the number of components in the MDT model) as inputs. It initializes a Gaussian Mixture Model (GMM) with n_components=num_components and fits it to the features using the fit() method. The trained model is then returned.
The scikit-learn library installed (pip install scikit-learn) before running this code.

In [3]:
import numpy as np
from sklearn.mixture import GaussianMixture

def train_mdt_model(features, num_components):
    # Convert the list of features into a 3D numpy array
    features_array = np.array(features)
    
    # Get the shape of the array
    num_samples, num_frames, num_features = features_array.shape
    
    # Reshape the array to have two dimensions
    features_2d = features_array.reshape(num_samples * num_frames, num_features)
    
    # Train a Gaussian Mixture Model (GMM) on the extracted features
    gmm = GaussianMixture(n_components=num_components)
    gmm.fit(features_2d)
    
    return gmm

In [4]:
def compute_anomaly_score(observation, model):
    # Compute the anomaly score by comparing the observation with the learned model
    score = -model.score_samples(observation.reshape(1, -1))
    return score

In [5]:
import os
import tifffile

# Specify the directory path containing the frames
directories =[
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train001",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train002",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train003",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train004",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train005",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train006",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train007",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train009",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train010",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train011",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train012",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train013",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train014",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train015",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train016",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train017",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train018",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train019",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train020",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train020",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train021",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train022",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train023",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train024",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train025",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train026",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train027",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train028",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train029",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train031",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train032",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train033",
    r"D:\Data Set\Crowd Anomaly Detection\UCSD_Anomaly_Dataset\UCSDped1\Train\Train034"
    
]
# Initialize an empty list to store the frames
frames = []

# Iterate over the directories
for directory_path in directories:
    # Iterate over the files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Check if the file is a .tif image
        if filename.endswith(".tif"):
            # Read the frame using tifffile
            frame = tifffile.imread(file_path)

            # Append the frame to the list
            frames.append(frame)

# Print the number of frames loaded
print("Number of frames:", len(frames))

Number of frames: 7000


In [6]:
# Initialize variables
num_components = 10  # Number of components in the MDT model
features = []

In [7]:
def extract_features(frames):
    features = []

    # Calculate optical flow for consecutive frames
    for i in range(len(frames) - 1):
        frame1 = frames[i]
        frame2 = frames[i + 1]

        # Calculate optical flow
        flow = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Flatten the flow matrix and append it to the features
        flow_flat = flow.reshape(-1)
        features.append(flow_flat)

    return features

In [8]:
# Create an empty list to store the features
features = []

# Iterate over the frames and extract features
for frame in frames:
    # Preprocess frame if needed
    # Extract features from the frame
    frame_features = extract_features(frame)
    features.append(frame_features)

In [9]:
# Convert features to numpy array
features = np.array(features)

In [10]:
import numpy as np

# Convert the list of features into a 3D numpy array
features_array = np.array(features)

# Get the shape of the array
num_samples, num_frames, num_features = features_array.shape

# Reshape the array to have two dimensions
features_2d = features_array.reshape(num_samples * num_frames, num_features)

In [11]:
# Train MDT model
mdt_model = train_mdt_model(features, num_components)

In [1]:
import cv2
import numpy as np

# Load the new frame using OpenCV
new_frame = cv2.imread("D:/Data Set/Crowd Anomaly Detection/UCSD_Anomaly_Dataset/UCSDped1/Test/Test034/023.tif")

# Check if the new frame is loaded successfully
if new_frame is None:
    print("Failed to load the new frame.")
    exit()

# Extract features from the new frame
new_features = extract_features([new_frame])  # Pass the new frame as a list to the extract_features function

# Check if the features are extracted successfully
if len(new_features) == 0:
    print("Failed to extract features from the new frame.")
    exit()

# Convert the features to a numpy array
new_features = np.array(new_features)

# Reshape the features for prediction
new_observation = new_features.reshape(1, -1)

# Compute anomaly score for the new observation
anomaly_score = compute_anomaly_score(new_observation, mdt_model)

NameError: name 'extract_features' is not defined

In [ ]:
print("Anomaly Score:", anomaly_score)

### *Conclusion:* 
Based on the provided code and information, the recommended model for anomaly detection in crowd behavior is the MDT (Motion Density Texture) model trained using Gaussian Mixture Model (GMM). This model utilizes optical flow features extracted from consecutive frames to learn patterns and detect anomalies in crowd behavior.

## 4 - Saving the model
Save the DS best model in the Jupyter notebook `model.ipynb` in the following formats:

- `network.save('model.h5')` #keras

In [ ]:
mdt_model.save('model.h5')